In [1]:
def caesar_cipher(text, shift, mode='encrypt'):
    if mode == 'decrypt':
        shift = -shift
    result = []
    for char in text:
        if char.isalpha():
            start = ord('A') if char.isupper() else ord('a')
            new_char = chr((ord(char) - start + shift) % 26 + start)
            result.append(new_char)
        else:
            result.append(char)
    return ''.join(result)

name = 'Doskhan'
shift = 8

print(caesar_cipher(name, shift=shift))


Lwaspiv


In [5]:
class DES:
    def __init__(self, key):
        if len(key) != 8:
            raise ValueError("Key must be exactly 8 bytes long.")
        self.key = key

    def _permute(self, block, table):
        if len(block) * 8 != len(table):
            raise ValueError("Permutation table does not match block size.")
        return bytes([block[(i - 1) // 8] for i in table])

    def _generate_subkeys(self):
        return [self.key] * 16  # Simplified subkey generation for this example

    def _feistel_function(self, half_block, subkey):
        return bytes([b ^ s for b, s in zip(half_block, subkey[:4])])

    def _process_block(self, block, subkeys, reverse=False):
        initial_perm_table = [58, 50, 42, 34, 26, 18, 10, 2,
                              60, 52, 44, 36, 28, 20, 12, 4,
                              62, 54, 46, 38, 30, 22, 14, 6,
                              64, 56, 48, 40, 32, 24, 16, 8,
                              57, 49, 41, 33, 25, 17, 9, 1,
                              59, 51, 43, 35, 27, 19, 11, 3,
                              61, 53, 45, 37, 29, 21, 13, 5,
                              63, 55, 47, 39, 31, 23, 15, 7]
        final_perm_table = [40, 8, 48, 16, 56, 24, 64, 32,
                            39, 7, 47, 15, 55, 23, 63, 31,
                            38, 6, 46, 14, 54, 22, 62, 30,
                            37, 5, 45, 13, 53, 21, 61, 29,
                            36, 4, 44, 12, 52, 20, 60, 28,
                            35, 3, 43, 11, 51, 19, 59, 27,
                            34, 2, 42, 10, 50, 18, 58, 26,
                            33, 1, 41, 9, 49, 17, 57, 25]

        block = self._permute(block, initial_perm_table)
        left, right = block[:4], block[4:]

        if reverse:
            subkeys = subkeys[::-1]

        for subkey in subkeys:
            temp = right
            right = bytes([l ^ f for l, f in zip(left, self._feistel_function(right, subkey))])
            left = temp

        block = right + left
        return self._permute(block, final_perm_table)

    def encrypt(self, plaintext):
        if len(plaintext) % 8 != 0:
            raise ValueError("Plaintext must be a multiple of 8 bytes.")

        subkeys = self._generate_subkeys()
        ciphertext = b''
        for i in range(0, len(plaintext), 8):
            block = plaintext[i:i + 8]
            ciphertext += self._process_block(block, subkeys)
        return ciphertext

    def decrypt(self, ciphertext):
        if len(ciphertext) % 8 != 0:
            raise ValueError("Ciphertext must be a multiple of 8 bytes.")

        subkeys = self._generate_subkeys()
        plaintext = b''
        for i in range(0, len(ciphertext), 8):
            block = ciphertext[i:i + 8]
            plaintext += self._process_block(block, subkeys, reverse=True)
        return plaintext


# Provided key and plaintext
key = b'\x9a\x9d\xcal"\xedP '
plaintext = b'T\x9a\xd5\xdaw\xad6j'

# Create DES instance
des = DES(key)

decrypted_text = des.decrypt(plaintext)
print(f"Decrypted: {decrypted_text}")



Decrypted: b'\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO\xda*\xd5~\x9a\xfdTO'


In [ ]:

key = b'\x9a\x9d\xcal"\xedP '
plaintext = b'T\x9a\xd5\xdaw\xad6j'

# Create DES instance
des = DES(key)

decrypted_text = des.decrypt(plaintext)
print(f"Decrypted: {decrypted_text}")


In [6]:
import random
from math import gcd

# Function to calculate modular inverse
def mod_inverse(a, m):
    # Extended Euclidean Algorithm to find modular inverse
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1

# Function to generate primes using Miller-Rabin primality test
def is_prime(n, k=5):  # Number of tests = k
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

# Function to generate a large prime number
def generate_prime(bits=1024):
    while True:
        prime_candidate = random.getrandbits(bits)
        if is_prime(prime_candidate):
            return prime_candidate

# Function to generate RSA keys
def generate_RSA_keys(bits=1024):
    p = generate_prime(bits // 2)  # Generate first prime p
    q = generate_prime(bits // 2)  # Generate second prime q
    while p == q:  # Ensure p != q
        q = generate_prime(bits // 2)
    
    n = p * q  # Modulus
    phi_n = (p - 1) * (q - 1)  # Euler's totient function

    # Find public key e such that gcd(e, phi_n) = 1
    e = 65537  # Common choice for e
    while gcd(e, phi_n) != 1:
        e = random.randrange(2, phi_n)

    # Find private key d such that (d * e) % phi_n = 1
    d = mod_inverse(e, phi_n)

    return (e, n), (d, n)

# Function to encrypt a message
def encrypt(public_key, message):
    e, n = public_key
    # Convert the message to integers (e.g., using ord())
    message_integers = [ord(char) for char in message]
    # Encrypt each integer
    encrypted_message = [pow(m, e, n) for m in message_integers]
    return encrypted_message

# Function to decrypt a message
def decrypt(private_key, encrypted_message):
    d, n = private_key
    # Decrypt each encrypted integer
    decrypted_message = [chr(pow(m, d, n)) for m in encrypted_message]
    return ''.join(decrypted_message)


In [10]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Generate RSA Keys
def generate_RSA_keys(bits=2048):
    key = RSA.generate(bits)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return public_key, private_key

# Encrypt a message
def encrypt(public_key, message):
    public_key = RSA.import_key(public_key)
    cipher = PKCS1_OAEP.new(public_key)
    encrypted_message = cipher.encrypt(message.encode())
    return b64encode(encrypted_message).decode()

# Decrypt a message
def decrypt(private_key, encrypted_message):
    private_key = RSA.import_key(private_key)
    cipher = PKCS1_OAEP.new(private_key)
    encrypted_message = b64decode(encrypted_message)
    decrypted_message = cipher.decrypt(encrypted_message).decode()
    return decrypted_message

# Example usage
if __name__ == "__main__":
    # Generate RSA keys
    public_key, private_key = generate_RSA_keys()

    # Original message
    message = "Hello, RSA with PyCryptodome!"

    # Encrypt the message
    encrypted_message = encrypt(public_key, message)
    print("Encrypted Message:", encrypted_message)

    # Decrypt the message
    decrypted_message = decrypt(private_key, encrypted_message)
    print("Decrypted Message:", decrypted_message)


ModuleNotFoundError: No module named 'Crypto'

In [9]:
!pip install pycryptodome



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
public_key, private_key = generate_RSA_keys(4096)
message = 'Doskhan'
encrypted_message = encrypt(public_key, message)
print("Encrypted Message:", encrypted_message)

# Decrypt the message
decrypted_message = decrypt(private_key, encrypted_message)
print("Decrypted Message:", decrypted_message)

KeyboardInterrupt: 

In [1]:
import pandas as pd

df = pd.read_csv('../parsing_results/output_links.csv')
df.head()

,Link
0,https://krisha.kz/prodazha/uchastkov/?page=1/a...
1,https://krisha.kz/prodazha/uchastkov/?page=1/a...
2,https://krisha.kz/prodazha/uchastkov/?page=1/a...
3,https://krisha.kz/prodazha/uchastkov/?page=1/a...
4,https://krisha.kz/prodazha/uchastkov/?page=1/a...


In [4]:
new_df = {
    "Link": []
}

for link in list(df["Link"]):
    link = link.split('/')
    new_df["Link"].append('/'.join(link[:3] + link[6:]))
    
new_df = pd.DataFrame(new_df)
new_df.iloc[0, 0]

'https://krisha.kz/a/show/1000787198?srchid=0196c647-e5bb-75dd-a15b-0191f6e31d91&srchtype=hot_block_filter&srchpos=3'

In [5]:
new_df.to_csv('../parsing_results/output_links.csv', index=False)